In [1]:
import pandas as pd 
import time
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import random
import numpy as np
import itertools
import math
from itertools import chain
import operator


In [2]:
def task_reward(members, competency_lists, request):
    # task utility is the sum the competency of coalition members.
    # request: task requested capabilities
    if members == []:
        return 0
    else:
        coal_pro = [competency_lists[mem] for mem in members]
        # return sum([sum([agent_pro[cap] for agent_pro in coal_pro]) for cap in request])
        return sum([max([agent_pro[cap] for agent_pro in coal_pro]) for cap in request])

def sys_reward(CS, competency_lists, t_caps_lists):
    ini_task_u = [[] for j in range(0, len(CS)-1)]
    for j in range(0, len(CS)-1):
        members = CS[j]
        if not members:
            ini_task_u[j] = 0
        else:
            ini_task_u[j] = task_reward(members, competency_lists, t_caps_lists[j])
    global_u = sum(ini_task_u)
    return global_u


def agent_con(competency_lists, t_caps_lists, query_agentIndex, query_taskIndex, member, a_taskInds):
    # the marginal contribution of an agent to a coalition

    if query_taskIndex == len(t_caps_lists):
        return 0
    if query_taskIndex not in a_taskInds[query_agentIndex]:
        return 0
    cur_reward = task_reward(member, competency_lists, t_caps_lists[query_taskIndex])
    if query_agentIndex in member:
        remained_mems = member[:]
        remained_mems.remove(query_agentIndex)  # the remained coalition members besides of agent i
        new_reward = task_reward(remained_mems, competency_lists, t_caps_lists[query_taskIndex])
        return cur_reward - new_reward
    else:
        added_mems = member[:]
        added_mems.append(query_agentIndex)
        new_reward = task_reward(added_mems, competency_lists, t_caps_lists[query_taskIndex])
        return new_reward - cur_reward

In [3]:
# def generate_noise(mean = 0, sd = 1): # generate a gussian noise, by defult 10% variation
#     return np.random.normal(mean,sd)


# Functions for generating environment with capability
# generating task
def gen_tasks(task_num, max_capNum, capabilities): # n is the number of task, max_capNum is the maximum number of cap a task could require
    return [sorted(np.random.choice(capabilities, np.random.randint(1, max_capNum+1),replace=False)) for j in range(0, task_num)]



def gen_agents(a_taskInds, tasks, max_capNum, capabilities,
               max_capVal):  # m is the number of task, max_capNum is the maximum number of cap a task could require, max_capVal is the maximum capability value

    # tasks: the lists of caps requested by tasks
    agent_num = len(a_taskInds)
    caps_lists = []
    contri_lists = []
    for i in range(0, agent_num):
        t_caps = [tasks[j] for j in a_taskInds[i]]  # lists of caps that each agent could perform

        caps_union = set(itertools.chain(*t_caps))  # union of caps of tasks that agent could perform
        a_cap_num = np.random.randint(min(1, max_capNum, len(caps_union)),
                                      min(len(caps_union), max_capNum) + 1)  # the num of caps the agent will have

        a_caps = set([np.random.choice(t_c) for t_c in
                      t_caps])  # initial draw to guarantee the agent has some contribution to each of the task that he could do

        rest_choices = list(caps_union.difference(a_caps))
        if rest_choices != []:
            update_len = max(0, a_cap_num - len(a_taskInds[i]))
            a_caps.update(np.random.choice(rest_choices, min(len(rest_choices), update_len), replace=False))

        caps_lists.append(sorted(list(a_caps)))

        contri_lists.append(
            [(np.random.randint(1, max_capVal + 1) if c in caps_lists[i] else 0) for c in range(0, len(capabilities))])
    
#     # generating agent costs
#     a_val = [sum(contri_lists[i]) for i in range(0, agent_num)] # Total cap valus of agents
#     a_maxVal = max(a_val) # max total cap value among agents
#     a_minVal = min(a_val) # min total cap value among agents

#     a_com = [1.0*(a_val[i]-a_minVal)/(a_maxVal-a_minVal) for i in range(0, agent_num)] # min-max scaling to [0,1]
#     a_com = [min(1,max(0,a_com[i]+np.random.normal(0,sd))) for i in range(0, agent_num)] # introduce random noise to each agent
    
#     a_costs = [int(p_min+a_com[i]*(p_max-p_min))  for i in range(0, agent_num)] # min-max scaling to [p_min, p_max]]

    return caps_lists, contri_lists


def random_in_budget(a_tasks, a_costs, task_num, B):
    agent_num = len(a_tasks)
    ava_agents = list(range(0, agent_num))
    alloc = [task_num]*agent_num
    sys_cost = 0
    while True: 
        selec_a = np.random.choice(ava_agents)
        sys_cost += a_costs[selec_a]
        if sys_cost > B:
            break
        else:
            ava_tasks = list(filter(lambda x: x != task_num, a_tasks[selec_a]))
            if ava_tasks==[]:
                ava_tasks = task_num                
            selec_t = np.random.choice(ava_tasks)
            alloc[selec_a] = selec_t
            ava_agents.remove(selec_a)
            if not ava_agents:
                break
    return alloc


def gen_costs(contri_lists, p_min = 1, p_max=20, mean=0, sd=0.1):
    # generating agent costs
    a_val = [sum(contri_lists[i]) for i in range(0, agent_num)] # Total cap valus of agents
    # print("a_val",a_val)
    a_maxVal = max(a_val) # max total cap value among agents
    a_minVal = min(a_val) # min total cap value among agents

    a_com = [1.0*(a_val[i]-a_minVal)/(a_maxVal-a_minVal) for i in range(0, agent_num)] # min-max scaling to [0,1]
    a_com = [min(1,max(0,a_com[i]+np.random.normal(mean,sd))) for i in range(0, agent_num)] # introduce random noise to each agent

    
    a_costs = [int(p_min+a_com[i]*(p_max-p_min))  for i in range(0, agent_num)] # min-max scaling to [p_min, p_max]]
    return a_costs
    


In [4]:
def cal_MV(cur_con, t_con, assigned = False, cost = 0, algo = 'GNE'):
    if algo == 'GNE':
        return t_con - cur_con  # GreedyNE moveval
    elif assigned: # CF movement values
        return (int(t_con) - int(cur_con))* (10 ** 6)  # the agent has been assigned to a task
    else:
        return 1.0 * (t_con - cur_con) / cost  # the agent has not been assigned to a task
        

def choose_movement(move_vals, agent_set):
    max_vals = [max(move_vals[i]) for i in agent_set]
    v_index, max_v = max(enumerate(max_vals), key=operator.itemgetter(1))
    a_index = agent_set[v_index]
    t_index = move_vals[a_index].index(max_v)
    return a_index, t_index, max_v


def TA_func(t_caps_lists, competency_lists, a_tasks, t_agents, budget, a_costs, cs=[], cur_cost=0, algo='GNE', time_bound=math.inf):  # GNE: GreedyNE; CF:cost efficiency

    start_time = time.time()
    agent_num = len(competency_lists)
    task_num = len(t_caps_lists)

    if cs == []:  # start from scratch
        cs = [[] for j in range(0, task_num + 1)]  # current coalition structure, the last one is dummy coalition
        cs[task_num] = list(range(0, agent_num))
        unassig_agents = list(range(0, agent_num))
        assign_agents = []
        feasible_agents = [i for i in unassig_agents if a_costs[i] <= budget]
        state = [task_num] * agent_num  # each indicate the current task that agent i is allocated to, if = N, means not allocated
        assigned = [False] * agent_num  # indicate whether an agent has been assigned, same as state, but more efficient for some computation
        cur_con = [0] * agent_num
        task_cons = [[agent_con(competency_lists, t_caps_lists, i, j, [], a_tasks) \
                      for j in range(0, task_num + 1)] \
                     for i in range(0, agent_num)]
    else:
        state = [task_num] * agent_num
        assigned = [False] * agent_num
        for j in range(0, task_num):
            for i in cs[j]:
                state[i] = j
                assigned[i] = True
        unassig_agents = cs[task_num][:]
        assign_agents = list(chain.from_iterable([cs[j] for j in range(0, task_num)]))
        feasible_agents_un = [i for i in unassig_agents if a_costs[i] <= budget]
        feasible_agents = assign_agents+feasible_agents_un
        cur_con = [agent_con(competency_lists, t_caps_lists, i, state[i], cs[state[i]], a_tasks) for i in range(0, agent_num)]
        task_cons = [[agent_con(competency_lists, t_caps_lists, i, j, cs[j], a_tasks) \
                      for j in range(0, task_num + 1)] \
                     for i in range(0, agent_num)]

    move_vals = [[cal_MV(cur_con[i], task_cons[i][j], assigned[i], a_costs[i], algo) \
                  if (cur_cost + a_costs[i] <= budget) else 0 for j in range(0, task_num + 1)]
                 for i in range(0, agent_num)]

    iteration = 0
    re_assign = 0
    is_continue = True

    while is_continue:
        if time.time() - start_time > time_bound:
            break
        
        iteration += 1
        if move_vals == []:
            break
        a_index, t_index, max_v = choose_movement(move_vals, feasible_agents)

        if max_v <= 0:
            is_continue = False
        else:
            # perform move
            # remove agent from old task
            old_t_index = state[a_index]
            cs[old_t_index].remove(a_index)
            # add agent to new task
            state[a_index] = t_index
            cs[t_index].append(a_index)

            # recording task that are affected
            affected_t_indexes = []
            affected_t_indexes.append(t_index)
            if old_t_index != task_num:  # agent has been assigned to a task before
                affected_t_indexes.append(old_t_index)
                re_assign += 1
                if t_index == task_num:  # agent goes back to dummy
                    cur_cost -= a_costs[a_index]
                    affected_t_indexes.remove(t_index)
                    unassig_agents.append(a_index)
                    assign_agents.remove(a_index)
                    assigned[a_index] = False
            else:  # and old_t_index == task_num, agent move from dummy to a task
                cur_cost += a_costs[a_index]
                assign_agents.append(a_index)
                unassig_agents.remove(a_index)
                assigned[a_index] = True

            unfeasible_agents = []  # [i for i in unassig_agents if cur_cost + a_costs[i] > budget]
            fea_agents = []
            for i in unassig_agents:
                if cur_cost + a_costs[i] <= budget:
                    fea_agents.append(i)
                else:
                    unfeasible_agents.append(i)

            for j in range(0, task_num):
                for i in unfeasible_agents:
                    move_vals[i][j] = 0

            feasible_agents = assign_agents + fea_agents
            # contribution of agents to tasks
            for j in affected_t_indexes:
                for i in t_agents[j]:
                    if i in feasible_agents:
                        cur_con[i] = agent_con(competency_lists, t_caps_lists, i, state[i], cs[state[i]], a_tasks)
                        task_cons[i][j] = agent_con(competency_lists, t_caps_lists, i, j, cs[j], a_tasks)
                        move_vals[i][j] = cal_MV(cur_con[i], task_cons[i][j], assigned[i], a_costs[i], algo)
                        # notice that agents in affected_a_indexes has already been assigned, so don't need to check feasibility
            affected_a_indexes = list(chain.from_iterable([cs[j] for j in affected_t_indexes]))

            for i in affected_a_indexes:
                move_vals[i] = [cal_MV(cur_con[i], task_cons[i][j], assigned[i], a_costs[i], algo) for j in
                                range(0, task_num + 1)]
    sys_u = sys_reward(cs, competency_lists, t_caps_lists)

    return sys_u, cs, cur_cost, iteration




In [5]:
# def choose_swap(swap_vals, assign_agents, unassig_agents):
#     max_vals = [max(sv) for sv in swap_vals]
#     av_index, max_v = max(enumerate(max_vals), key=operator.itemgetter(1)) # av_index the max agent's index in assign_agents
#     a_index = assign_agents[av_index]
#     uv_index = swap_vals[av_index].index(max_v) #uv_index the swaping agent's index in unassign_agents
#     u_index = unassig_agents[uv_index]
#     return av_index, uv_index, a_index, u_index, max_v


# def swap_deass(t_caps_lists, competency_lists, a_tasks, budget, a_costs, cs, state, assign_agents, unassig_agents, cur_cost):
#     task_num = len(t_caps_lists)
#     # cs, state, assign_agents = deassign(t_caps_lists, competency_lists, a_tasks, a_costs, cs, state, assign_agents, assign_agents, cur_cost)
#     ass_efficiency = [agent_con(competency_lists, t_caps_lists, a, state[a], cs[state[a]], a_tasks)/a_costs[a] for a in assign_agents]
#     zero_eff = np.where(np.array(ass_efficiency) == 0)  # the index of the agent in the assigned_agent set
#     zero_eff_agents = zero_eff[0]  # store the index of the non-contribution agents
#     de_assign_agents = []
#     for k in zero_eff_agents:
#         de_assign_agents.append(assign_agents[k])
#     if len(zero_eff_agents) != 0:  # de-assign non-contribution agents
#         for k in de_assign_agents:
#             cur_cost -= a_costs[k]
#             org_task = state[k]
#             cs[org_task].remove(k)
#             cs[task_num].append(k)
#             state[k] = task_num
#             assign_agents.remove(k)
#             unassig_agents.append(k)

#     other_members = [list(filter(lambda x: x != a, cs[state[a]])) for a in assign_agents]
#     swap_vals = [[task_reward(other_members[ind] + [u], competency_lists, t_caps_lists[state[a]]) -
#                   task_reward(cs[state[a]], competency_lists, t_caps_lists[state[a]]) \
#                       if (cur_cost - a_costs[a] + a_costs[u] <= budget) and (state[a] in a_tasks[u]) else 0
#                   for u in unassig_agents]
#                  for ind, a in enumerate(assign_agents)]
    
#     is_continue = True
#     start_time = time.time()
#     while is_continue:
#         if time.time() - start_time > 300:
#             break
#         av_index, uv_index, a_index, u_index, max_v = choose_swap(swap_vals, assign_agents, unassig_agents)
#         if max_v <= 0:
#             is_continue = False
#         else:
#             # perform swap
#             # remove agent a from task to dummy
#             cur_cost -= a_costs[a_index]
#             cur_cost += a_costs[u_index]
#             t_index = state[a_index]
#             state[a_index] = task_num
#             cs[t_index].remove(a_index)
#             cs[task_num].append(a_index)

#             # add agent u to task
#             state[u_index] = t_index
#             cs[t_index].append(u_index)
#             cs[task_num].remove(u_index)

#             # swap a and u in assign_agents and unassig_agents
#             assign_agents[av_index] = u_index
#             unassig_agents[uv_index] = a_index

#             other_members = [list(filter(lambda x: x != a, cs[state[a]])) for a in assign_agents]
#             swap_vals = [[task_reward(other_members[ind] + [u], competency_lists, t_caps_lists[state[a]]) -
#                           task_reward(cs[state[a]], competency_lists, t_caps_lists[state[a]]) \
#                               if (cur_cost - a_costs[a] + a_costs[u] <= budget) and (state[a] in a_tasks[u]) else 0
#                           for u in unassig_agents]
#                          for ind, a in enumerate(assign_agents)]

#     sys_u = sys_reward(cs, competency_lists, t_caps_lists)
#     return sys_u, cs, cur_cost




In [6]:
########## general GA ##############
# def random_good_initial(state):
#     new_state = state[:]
#     select_ind = np.random.choice(list(range(0, len(state))), size=2, replace=False)
#     new_state[select_ind[0]] = state[select_ind[1]]
#     new_state[select_ind[1]] = state[select_ind[0]]
            
#     return new_state

# fitness
def fitness_GA(CS, competency_lists, t_caps_lists):
    ini_task_u = [[] for j in range(0, len(CS)-1)]
    for j in range(0, len(CS)-1):
        members = CS[j]
        if not members:
            ini_task_u[j] = 0
        else:
            ini_task_u[j] = task_reward(members, competency_lists, t_caps_lists[j])
    global_u = sum(ini_task_u)
    return global_u #, ini_task_u


def state_u_c(alloc, competency_lists, t_caps_lists, a_costs, B):
    task_num = len(t_caps_lists)
    CS = [[] for j in range(0, task_num+1)]
    sys_cost = 0
    for i in range(0,agent_num):
        CS[alloc[i]].append(i)
        if alloc[i] != task_num:
            sys_cost += a_costs[i]        
    sys_u = fitness_GA(CS, competency_lists, t_caps_lists) if sys_cost <= B else 0    
    return sys_u


def gen_population(population_size, competency_lists, t_caps_lists, a_costs, a_tasks, B, ini_popu=[]):
    task_num = len(t_caps_lists)
    population = {}
    for k in range(0, population_size - len(ini_popu)):
        state = random_in_budget(a_tasks, a_costs, task_num, B)
        sys_u = state_u_c(state, competency_lists, t_caps_lists, a_costs, B)
        population['inidi' + str(k)] = state
        population['fitness' + str(k)] = sys_u
    for j in range(0, len(ini_popu)):
        population['inidi' + str(population_size - j - 1)] = ini_popu[j]['inidi'][:]
        population['fitness' + str(population_size - j - 1)] = ini_popu[j]['fitness']
    return population


def GA_func(competency_lists, t_caps_lists, a_costs, a_tasks, B, population, pc, pm, m_points, iter_bound, time_bound):

    agent_num = len(competency_lists)
    task_num = len(t_caps_lists)
    population_size = int(len(population) / 2)

    start_time = time.time()
    iter = 1
    record_u = []
    while True:  # iter <= iter_bound:  #
        if time.time() - start_time > time_bound:
            break
        if iter > iter_bound:
            if record_u[-1]-record_u[-iter_bound]==0:
                break
        ############ selection ###################
        current_fit = []
        for k in list(range(0, population_size)):
            current_fit.append(population['fitness' + str(k)])
        weight = [current_fit[i] / sum(current_fit) for i in range(0, population_size)]

        ############ generate new poputation ############
        new_population = {}
        new_population_size = 0
        while new_population_size < population_size:
            select_parents = np.random.choice(list(range(0, population_size)), size=2, replace=False, p=weight)
            parent0 = population['inidi' + str(select_parents[0])][:]
            parent1 = population['inidi' + str(select_parents[1])][:]
            ########### start crossover ################
            R_rand = np.random.random(1)
            if R_rand <= pc:
                crossover_point = np.random.choice(list(range(0, agent_num)))
                new_population['inidi' + str(new_population_size)] = [parent0[i] if i <= crossover_point else
                                                                      parent1[i] for i in range(0, agent_num)]
                new_population['inidi' + str(new_population_size + 1)] = [parent1[i] if i <= crossover_point else
                                                                          parent0[i] for i in range(0, agent_num)]
            else:
                new_population['inidi' + str(new_population_size)] = parent0[:]
                new_population['inidi' + str(new_population_size + 1)] = parent1[:]
            ########### end crossover ################

            ########### start mutation ################
            M_rand = np.random.random(2)
            index_k = 0
            for k in list(range(0, 2)):
                if M_rand[index_k] <= pm:
                    mutation_points = list(
                        np.random.choice(list(range(0, agent_num)), m_points, replace=False)) if m_points != 1 else [
                        np.random.choice(list(range(0, agent_num)))]  # randomly generate the mutation points.
                    for m in range(0, m_points):
                        org_gene = new_population['inidi' + str(new_population_size + k)][mutation_points[m]]
                        other_genes = list(filter(lambda x: x != org_gene, a_tasks[mutation_points[m]]))
                        new_population['inidi' + str(new_population_size + k)][mutation_points[m]] = np.random.choice(
                            other_genes)
                new_population['fitness' + str(new_population_size + k)] = state_u_c(new_population['inidi' + str(new_population_size + k)], competency_lists, t_caps_lists, a_costs, B)
                index_k += 1
                ########### end mutation ################
            new_population_size += 2

        ########## strat update population ################
        new_generation_fit = []
        for k in list(range(0, population_size)):
            new_generation_fit.append(new_population['fitness' + str(k)])

        compare_fitness = current_fit + new_generation_fit
        rank_com_ind = [index for index, value in
                        sorted(list(enumerate(compare_fitness)), key=lambda x: x[1], reverse=True)]
        for k in list(range(0, population_size)):
            if rank_com_ind[k] < population_size:
                population['inidi' + str(k)] = population['inidi' + str(rank_com_ind[k])][:]
                population['fitness' + str(k)] = population['fitness' + str(rank_com_ind[k])]
            else:
                population['inidi' + str(k)] = new_population['inidi' + str(rank_com_ind[k] - population_size)][:]
                population['fitness' + str(k)] = new_population['fitness' + str(rank_com_ind[k] - population_size)]
        ########## end update population ################
        iter += 1
        record_u.append(population['fitness' + str(0)])
        ############### results ######################
    GA_solution = population['inidi' + str(0)]
    GA_u = population['fitness' + str(0)]
    GA_cost = 0
    for i in range(0, agent_num):
        if GA_solution[i] != task_num:
            GA_cost += a_costs[i]
    return GA_u, GA_solution, GA_cost, iter-1, record_u


In [7]:
class cartesian(object):
    def __init__(self):
        self._data_list=[]

    def add_data(self,data=[]): # add in data to generate the list of Cartesian
        self._data_list.append(data)

    def build(self): # calculate Cartesian
        all_combination = []
        for item in itertools.product(*self._data_list):
            all_combination.append(list(item))
        return all_combination


def combination(combine_list):
    # combine_list: the domain of agents
    car = cartesian()
    for i in range(0, len(combine_list)):
        car.add_data(list(combine_list[i]))
    comb = car.build()
    return comb

def exhaustive_search(t_caps_lists, competency_lists, a_tasks, B, a_costs):
    task_num = len(t_caps_lists)
    domains = np.array(a_tasks)
    all_comb = combination(domains)
    all_results = []
    all_costs = []
    for i in range(0, len(all_comb)):
        CS = [[] for j in range(0, task_num + 1)]
        total_cost = 0
        ini_alloc = list(all_comb[i])  # agents' selections, an allocation solution
        selected_tasks = list(set(ini_alloc))
        dummy_index = np.where(np.array(selected_tasks) == task_num)
        if len(dummy_index[0]) != 0:
            del selected_tasks[int(dummy_index[0])]
        for j in selected_tasks:
            members = np.where(np.array(ini_alloc) == j)
            CS[j] = list(members[0])
            task_cost = sum(a_costs[k] for k in CS[j])
            total_cost += task_cost
        if total_cost > B:
            reached_u = 0
        else:
            reached = sys_reward(CS, competency_lists, t_caps_lists)
            reached_u = reached[0]

        all_results.append(reached_u)
        all_costs.append(total_cost)
    optimal_u = max(all_results)
    index = np.where(np.array(all_results) == optimal_u)
    compare_cost = []
    for x in list(index[0]):
        compare_cost.append(all_costs[x])

    opt_cost = min(compare_cost)
    cost_min_ind = np.where(np.array(compare_cost) == opt_cost)
    index_com = cost_min_ind[0][0]
    opt_index = index[0][index_com]
    opt_structure = all_comb[opt_index]

    OPT_CS = [[] for j in range(0, task_num + 1)]
    allocation_str = list(set(opt_structure))
    for j in allocation_str:
        OPT_members = np.where(np.array(opt_structure) == j)
        OPT_CS[j] = list(OPT_members[0])

    return optimal_u, OPT_CS, opt_cost

In [8]:
def append_record(record, filename, typ):
    with open(filename, 'a') as f:
        if typ != '':
            json.dump(record, f, default = typ)
        else:
            json.dump(record, f)
        f.write('\n')
        # f.write(os.linesep)
        f.close()

In [9]:
run_num = 100
variation = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
task_num =100
agent_num = 3*task_num                    
B = 5*task_num   
capNum = 20
capabilities = list(range(0, capNum))
max_capNum_task = 20
max_capNum_agent = 20
max_capVal = 10
c_min = 1  # minimum cost
c_max = 20  # maximum cost
time_bound = 1000  # time bound for GA
iter_bound = 100 # math.inf  # the generation of offspring
population_size = 100  # the initial population size
pc = 0.9  # crossover rate
pm = 0.5  # mutation rate
m_points = 1  # number of mutation points
ex_identifier = 0


# compare algorithms
for run in range(0, run_num):
    print('run:', run)
    # generating tasks (representing as capabilities it needs)
    t_caps_lists = gen_tasks(task_num, max_capNum_task, capabilities)
    # randomly generate a list of tasks feasible for an agent (constraints); an agent can work on [1,task_num/2] tasks
    a_tasks = [list(np.random.choice(range(0, task_num), np.random.randint(1, task_num/2+1), replace=False))
               for i in range(0, agent_num)]

    # get also the list of agents for each task
    t_agents = [[i for i in range(0, agent_num) if j in a_tasks[i]] for j in range(0, task_num)]
    # generating agents, including their capabilities, competency values, and costs
    a_caps_lists, competency_lists = gen_agents(a_tasks, t_caps_lists, max_capNum_agent, capabilities,
                                                max_capVal)
    
    for i in range(0, agent_num):
        a_tasks[i].append(task_num)
    t_agents.append(list(range(0, agent_num)))

    for var in variation:
        print('variation: '+str(var))
        ex_identifier += 1
        a_costs = gen_costs(competency_lists, c_min, c_max, 0, var)
        
        result = {"ex_identifier": ex_identifier, "task_num": task_num, "agent_num": agent_num, "var": var}


        ##  a random solution
        start = time.time()
        rand_state = random_in_budget(a_tasks, a_costs, task_num, B)
        rand_CS = [[] for i in list(range(0, task_num + 1))]
        rand_cost = 0
        for i in list(range(0, agent_num)):
            if rand_state[i] != task_num:
                rand_cost += a_costs[i]
            rand_CS[rand_state[i]].append(i)
        random_u = sys_reward(rand_CS, competency_lists, t_caps_lists)        
        end = time.time()
        rand_allo_num = agent_num-len(rand_CS[task_num])
        result['random_u'] = random_u
        result['random_c'] = int(rand_cost)
        result['random_t'] = end - start
        result['rand_allo_num'] = rand_allo_num
        print("task number: ", task_num, 'random_u:', random_u,
              "random_c:", rand_cost, "random_t:", end - start, "rand_allo_num:", rand_allo_num)
        print()

        #  GreedyNE
        start = time.time()
        [GreedyNE_u, GreedyNE_CS, GreedyNE_cost, iter] = TA_func(t_caps_lists, competency_lists, a_tasks, t_agents,
                                                                 B, a_costs, [], 0, "GNE", time_bound)
        end = time.time()
        GreedyNE_allo_num = agent_num-len(GreedyNE_CS[task_num])
        result['GreedyNE_u'] = GreedyNE_u
        result['GreedyNE_c'] = int(GreedyNE_cost)
        result['GreedyNE_t'] = end - start
        result['GreedyNE_iter'] = iter
        result['GreedyNE_allo_num'] = GreedyNE_allo_num
        print("task number: ", task_num, 'GreedyNE_u:', GreedyNE_u,
              "GreedyNE_c:", GreedyNE_cost, "GreedyNE_t:", end - start, 'GreedyNE iteration:', iter, "GreedyNE_allo_num:", GreedyNE_allo_num)
        print()

        ##  CF
        start = time.time()
        [CF_u, CF_CS, CF_cost, CF_iter] = TA_func(t_caps_lists, competency_lists, a_tasks, t_agents, B, a_costs, [],
                                                  0, "CF", time_bound)
        end = time.time()
        CF_allo_num = agent_num-len(CF_CS[task_num])
        result['CF_u'] = CF_u
        result['CF_c'] = int(CF_cost)
        result['CF_t'] = end - start
        result['CF_iter'] = CF_iter
        result['CF_allo_num'] = CF_allo_num
        print("task number: ", task_num, 'CF utility:', CF_u,
              "CF cost:", CF_cost, "CF time:", end - start, 'CF iteration:', CF_iter, 'CF_allo_num:', CF_allo_num)
        print()

        # GA
        start = time.time()
        population = gen_population(population_size, competency_lists, t_caps_lists, a_costs, a_tasks, B, [])
        [GA_u, GA_solution, GA_cost, GA_iteration, GA_record] = GA_func(competency_lists, t_caps_lists, a_costs, a_tasks, B,
                                                             population, pc, pm, m_points, iter_bound, time_bound)
        end = time.time()
        GA_CS = [[] for j in range(0, task_num+1)] 
        for i in range(0, task_num):
            ind = GA_solution[i]
            GA_CS[ind].append(i)
        GA_allo_num = agent_num-len(GA_CS[task_num])    
        result['GA_u'] = GA_u
        result['GA_c'] = GA_cost
        result['GA_t'] = end - start
        result['GA_iter'] = GA_iteration
        result['GA_allo_num'] = GA_allo_num
        print("task number: ", task_num, 'GA utility:', GA_u,
              "GA cost:", GA_cost, "GA time:", end - start, 'GA iteration:', GA_iteration,"GA_allo_num:", GA_allo_num)
        print()

        take_ind = int(result['CF_t']*GA_iteration/result['GA_t'])          
        result['int_GA_u'] = GA_record[take_ind]
        result['int_GA_iter'] = take_ind
        print("task number: ", task_num, 'int_GA utility:', GA_record[take_ind], 'int_GA iteration:', take_ind)
        print()


        files = {'varying_variation': [result, '']}
        for filename in list(files.keys()):
            append_record(files[filename][0], filename, typ=files[filename][1])








run: 0
task number:  100 random_u: 1694 random_c: 495 random_t: 0.003989219665527344 rand_allo_num: 41

task number:  100 GreedyNE_u: 3063 GreedyNE_c: 500 GreedyNE_t: 0.28324294090270996 GreedyNE iteration: 31 GreedyNE_allo_num: 30

task number:  100 CF utility: 3111 CF cost: 500 CF time: 0.37949538230895996 CF iteration: 56 CF_allo_num: 54

task number:  100 GA utility: 2835 GA cost: 500 GA time: 17.69984006881714 GA iteration: 338 GA_allo_num: 214

task number:  100 int_GA utility: 1910 int_GA iteration: 0

task number:  100 random_u: 1678 random_c: 500 random_t: 0.0029916763305664062 rand_allo_num: 41

task number:  100 GreedyNE_u: 3119 GreedyNE_c: 499 GreedyNE_t: 0.29125070571899414 GreedyNE iteration: 31 GreedyNE_allo_num: 30

task number:  100 CF utility: 3689 CF cost: 499 CF time: 0.47672557830810547 CF iteration: 69 CF_allo_num: 63

task number:  100 GA utility: 2914 GA cost: 500 GA time: 21.966259241104126 GA iteration: 431 GA_allo_num: 215

task number:  100 int_GA utility: 1

task number:  100 GreedyNE_u: 3745 GreedyNE_c: 500 GreedyNE_t: 0.2952110767364502 GreedyNE iteration: 40 GreedyNE_allo_num: 39

task number:  100 CF utility: 5859 CF cost: 500 CF time: 0.8198230266571045 CF iteration: 166 CF_allo_num: 130

task number:  100 GA utility: 4006 GA cost: 500 GA time: 26.571195602416992 GA iteration: 466 GA_allo_num: 220

task number:  100 int_GA utility: 2001 int_GA iteration: 0

task number:  100 random_u: 1535 random_c: 500 random_t: 0.0029914379119873047 rand_allo_num: 45

task number:  100 GreedyNE_u: 3599 GreedyNE_c: 500 GreedyNE_t: 0.2892274856567383 GreedyNE iteration: 38 GreedyNE_allo_num: 37

task number:  100 CF utility: 5761 CF cost: 499 CF time: 0.7968840599060059 CF iteration: 162 CF_allo_num: 125

task number:  100 GA utility: 3520 GA cost: 500 GA time: 17.23408532142639 GA iteration: 307 GA_allo_num: 224

task number:  100 int_GA utility: 2092 int_GA iteration: 0

task number:  100 random_u: 1562 random_c: 486 random_t: 0.002991199493408203 r

task number:  100 CF utility: 7247 CF cost: 500 CF time: 0.911578893661499 CF iteration: 185 CF_allo_num: 151

task number:  100 GA utility: 4343 GA cost: 500 GA time: 26.0311758518219 GA iteration: 408 GA_allo_num: 221

task number:  100 int_GA utility: 2341 int_GA iteration: 0

run: 3
task number:  100 random_u: 1289 random_c: 498 random_t: 0.003989458084106445 rand_allo_num: 44

task number:  100 GreedyNE_u: 3060 GreedyNE_c: 500 GreedyNE_t: 0.27227210998535156 GreedyNE iteration: 30 GreedyNE_allo_num: 29

task number:  100 CF utility: 3195 CF cost: 500 CF time: 0.3789865970611572 CF iteration: 66 CF_allo_num: 65

task number:  100 GA utility: 2913 GA cost: 500 GA time: 13.126044034957886 GA iteration: 266 GA_allo_num: 216

task number:  100 int_GA utility: 1942 int_GA iteration: 0

task number:  100 random_u: 1528 random_c: 491 random_t: 0.0029909610748291016 rand_allo_num: 38

task number:  100 GreedyNE_u: 3143 GreedyNE_c: 500 GreedyNE_t: 0.27924513816833496 GreedyNE iteration: 31 

task number:  100 GA utility: 3410 GA cost: 500 GA time: 18.29426097869873 GA iteration: 351 GA_allo_num: 213

task number:  100 int_GA utility: 2194 int_GA iteration: 0

task number:  100 random_u: 1878 random_c: 492 random_t: 0.003989219665527344 rand_allo_num: 45

task number:  100 GreedyNE_u: 3749 GreedyNE_c: 500 GreedyNE_t: 0.3141603469848633 GreedyNE iteration: 36 GreedyNE_allo_num: 35

task number:  100 CF utility: 6109 CF cost: 499 CF time: 0.8776543140411377 CF iteration: 159 CF_allo_num: 121

task number:  100 GA utility: 4026 GA cost: 500 GA time: 20.946762323379517 GA iteration: 339 GA_allo_num: 216

task number:  100 int_GA utility: 2214 int_GA iteration: 0

task number:  100 random_u: 1767 random_c: 496 random_t: 0.002991914749145508 rand_allo_num: 47

task number:  100 GreedyNE_u: 4083 GreedyNE_c: 500 GreedyNE_t: 0.3061966896057129 GreedyNE iteration: 40 GreedyNE_allo_num: 39

task number:  100 CF utility: 6558 CF cost: 499 CF time: 0.7858984470367432 CF iteration: 162 C

task number:  100 GreedyNE_u: 3768 GreedyNE_c: 500 GreedyNE_t: 0.30019712448120117 GreedyNE iteration: 39 GreedyNE_allo_num: 38

task number:  100 CF utility: 7004 CF cost: 500 CF time: 0.8946280479431152 CF iteration: 185 CF_allo_num: 158

task number:  100 GA utility: 4829 GA cost: 500 GA time: 32.28478980064392 GA iteration: 454 GA_allo_num: 229

task number:  100 int_GA utility: 2389 int_GA iteration: 0

run: 6
task number:  100 random_u: 1460 random_c: 494 random_t: 0.0029914379119873047 rand_allo_num: 43

task number:  100 GreedyNE_u: 3206 GreedyNE_c: 500 GreedyNE_t: 0.28823065757751465 GreedyNE iteration: 31 GreedyNE_allo_num: 30

task number:  100 CF utility: 3204 CF cost: 499 CF time: 0.4218728542327881 CF iteration: 63 CF_allo_num: 58

task number:  100 GA utility: 2969 GA cost: 500 GA time: 20.221120595932007 GA iteration: 335 GA_allo_num: 210

task number:  100 int_GA utility: 1971 int_GA iteration: 0

task number:  100 random_u: 1758 random_c: 492 random_t: 0.0039882659912

...

task number:  100 GreedyNE_u: 3707 GreedyNE_c: 500 GreedyNE_t: 0.3001978397369385 GreedyNE iteration: 37 GreedyNE_allo_num: 36

task number:  100 CF utility: 6216 CF cost: 500 CF time: 0.7829239368438721 CF iteration: 154 CF_allo_num: 122

task number:  100 GA utility: 4469 GA cost: 500 GA time: 21.768513202667236 GA iteration: 367 GA_allo_num: 224

task number:  100 int_GA utility: 2209 int_GA iteration: 0

task number:  100 random_u: 1868 random_c: 500 random_t: 0.003988504409790039 rand_allo_num: 48

task number:  100 GreedyNE_u: 3795 GreedyNE_c: 500 GreedyNE_t: 0.3141601085662842 GreedyNE iteration: 38 GreedyNE_allo_num: 37

task number:  100 CF utility: 6609 CF cost: 500 CF time: 0.7729332447052002 CF iteration: 158 CF_allo_num: 132

task number:  100 GA utility: 4401 GA cost: 500 GA time: 34.07483673095703 GA iteration: 586 GA_allo_num: 224

task number:  100 int_GA utility: 2288 int_GA iteration: 0

task number:  100 random_u: 1508 random_c: 500 random_t: 0.002991199493408203 ra

task number:  100 CF utility: 6199 CF cost: 495 CF time: 0.825791597366333 CF iteration: 180 CF_allo_num: 155

task number:  100 GA utility: 4335 GA cost: 500 GA time: 32.97209620475769 GA iteration: 574 GA_allo_num: 228

task number:  100 int_GA utility: 2055 int_GA iteration: 0

run: 95
task number:  100 random_u: 1623 random_c: 484 random_t: 0.0029914379119873047 rand_allo_num: 42

task number:  100 GreedyNE_u: 3140 GreedyNE_c: 500 GreedyNE_t: 0.2732558250427246 GreedyNE iteration: 30 GreedyNE_allo_num: 29

task number:  100 CF utility: 3233 CF cost: 500 CF time: 0.33809638023376465 CF iteration: 52 CF_allo_num: 50

task number:  100 GA utility: 2949 GA cost: 500 GA time: 23.983188152313232 GA iteration: 500 GA_allo_num: 210

task number:  100 int_GA utility: 2109 int_GA iteration: 0

task number:  100 random_u: 1505 random_c: 488 random_t: 0.002992391586303711 rand_allo_num: 43

task number:  100 GreedyNE_u: 3215 GreedyNE_c: 500 GreedyNE_t: 0.2812473773956299 GreedyNE iteration: 31

task number:  100 GA utility: 3788 GA cost: 500 GA time: 31.696451902389526 GA iteration: 559 GA_allo_num: 224

task number:  100 int_GA utility: 2209 int_GA iteration: 0

task number:  100 random_u: 1781 random_c: 491 random_t: 0.002991199493408203 rand_allo_num: 47

task number:  100 GreedyNE_u: 4439 GreedyNE_c: 500 GreedyNE_t: 0.3171515464782715 GreedyNE iteration: 47 GreedyNE_allo_num: 46

task number:  100 CF utility: 6481 CF cost: 500 CF time: 0.7849133014678955 CF iteration: 161 CF_allo_num: 129

task number:  100 GA utility: 4473 GA cost: 500 GA time: 21.51120400428772 GA iteration: 364 GA_allo_num: 219

task number:  100 int_GA utility: 2280 int_GA iteration: 0

task number:  100 random_u: 1747 random_c: 500 random_t: 0.0029914379119873047 rand_allo_num: 49

task number:  100 GreedyNE_u: 4161 GreedyNE_c: 500 GreedyNE_t: 0.3041863441467285 GreedyNE iteration: 43 GreedyNE_allo_num: 42

task number:  100 CF utility: 6906 CF cost: 500 CF time: 0.8597011566162109 CF iteration: 177 

task number:  100 GreedyNE_u: 4059 GreedyNE_c: 500 GreedyNE_t: 0.3251311779022217 GreedyNE iteration: 42 GreedyNE_allo_num: 41

task number:  100 CF utility: 7185 CF cost: 494 CF time: 0.9045810699462891 CF iteration: 193 CF_allo_num: 157

task number:  100 GA utility: 4660 GA cost: 500 GA time: 29.275474309921265 GA iteration: 483 GA_allo_num: 226

task number:  100 int_GA utility: 2362 int_GA iteration: 0

run: 98
task number:  100 random_u: 1373 random_c: 495 random_t: 0.0029916763305664062 rand_allo_num: 42

task number:  100 GreedyNE_u: 3385 GreedyNE_c: 500 GreedyNE_t: 0.2842397689819336 GreedyNE iteration: 33 GreedyNE_allo_num: 32

task number:  100 CF utility: 3433 CF cost: 500 CF time: 0.40392184257507324 CF iteration: 64 CF_allo_num: 60

task number:  100 GA utility: 3134 GA cost: 500 GA time: 21.073838233947754 GA iteration: 435 GA_allo_num: 216

task number:  100 int_GA utility: 2135 int_GA iteration: 0

task number:  100 random_u: 1861 random_c: 497 random_t: 0.003988504409

task number:  100 CF utility: 5548 CF cost: 498 CF time: 0.7250823974609375 CF iteration: 141 CF_allo_num: 107

task number:  100 GA utility: 3765 GA cost: 500 GA time: 24.83078384399414 GA iteration: 455 GA_allo_num: 226

task number:  100 int_GA utility: 2026 int_GA iteration: 0

task number:  100 random_u: 1395 random_c: 497 random_t: 0.003988504409790039 rand_allo_num: 49

task number:  100 GreedyNE_u: 3550 GreedyNE_c: 500 GreedyNE_t: 0.28324151039123535 GreedyNE iteration: 36 GreedyNE_allo_num: 35

task number:  100 CF utility: 6068 CF cost: 499 CF time: 0.8258228302001953 CF iteration: 159 CF_allo_num: 125

task number:  100 GA utility: 4249 GA cost: 500 GA time: 35.364726543426514 GA iteration: 618 GA_allo_num: 230

task number:  100 int_GA utility: 2047 int_GA iteration: 0

task number:  100 random_u: 1647 random_c: 484 random_t: 0.0029909610748291016 rand_allo_num: 41

task number:  100 GreedyNE_u: 3997 GreedyNE_c: 500 GreedyNE_t: 0.3032040596008301 GreedyNE iteration: 43 Gree